# Robot Demo

This is the all-in-one demo:
- TTS (first heartbeat)
- Eyes RGB
- Camera head gestures
- Motors + drift
- Horn

If something fails, it prints the error and continues.


In [ ]:
# Bootstrap (shared)
from pathlib import Path
import os, sys

def _safe_start() -> Path:
    try:
        return Path.cwd()
    except FileNotFoundError:
        return Path(os.environ.get('HOME', '/tmp'))

p = _safe_start().resolve()
for _ in range(30):
    cand = p / 'common' / 'lib'
    if cand.is_dir():
        if str(cand) not in sys.path:
            sys.path.insert(0, str(cand))
        break
    if p.parent == p:
        break
    p = p.parent

import bootstrap
if hasattr(bootstrap, 'init'):
    bootstrap.init(default_domain='0', verbose=True)
else:
    bootstrap.bootstrap(default_domain='0', verbose=True)


In [ ]:
import importlib
import tts_lib
importlib.reload(tts_lib)

def speak(text: str):
    if hasattr(tts_lib, 'say'):
        return tts_lib.say(text)
    if hasattr(tts_lib, 'speak'):
        return tts_lib.speak(text)
    if hasattr(tts_lib, 'tts_to_wav'):
        wav = tts_lib.tts_to_wav(text)
        print('tts_to_wav ->', wav)
        return wav
    raise RuntimeError('No say()/speak()/tts_to_wav() found in tts_lib')

try:
    print('Testing TTS...')
    speak('Kia ora team. Robot demo starting now.')
    print('TTS OK')
except Exception as e:
    print('TTS failed:', e)


In [ ]:
import time, importlib
import robot_moves as rm
import eyes_lib
import camera_lib

importlib.reload(rm)
importlib.reload(eyes_lib)
importlib.reload(camera_lib)

bot = rm.RobotMoves(base_speed=220, rate_hz=30)
bot.stop(); time.sleep(0.2)
print('RobotMoves ready. BASE_SPEED=', rm.BASE_SPEED, 'RATE_HZ=', rm.RATE_HZ)

eyes = None
try:
    eyes = eyes_lib.get_eyes()
    if hasattr(eyes, 'diagnose'):
        eyes.diagnose()
    print('Eyes ready')
except Exception as e:
    print('Eyes init failed:', e)
    eyes = None

cam = None
try:
    cam = camera_lib.get_camera()
    print('Camera head ready')
except Exception as e:
    print('Camera head init failed:', e)
    cam = None


In [ ]:
import time
if not eyes:
    print('Skipping eyes demo (eyes not available)')
else:
    try:
        print('Eyes: red, green, blue, off')
        eyes.set_both(255, 0, 0); time.sleep(0.4)
        eyes.set_both(0, 255, 0); time.sleep(0.4)
        eyes.set_both(0, 0, 255); time.sleep(0.4)
        eyes.off(); time.sleep(0.2)
        print('Eyes: blink white')
        eyes.blink(color=(255,255,255), period_s=0.2, duration_s=1.2)
        print('Eyes demo done')
    except Exception as e:
        print('Eyes demo failed:', e)


In [ ]:
import time
if not cam:
    print('Skipping camera head demo (cam not available)')
else:
    try:
        print('Center...')
        cam.center_all()
        print('Nod...')
        cam.nod(depth=250)
        print('Shake...')
        cam.shake(width=250)
        print('Wiggle...')
        cam.wiggle(cycles=2, amplitude=200)
        print('Tiny wiggle...')
        cam.tiny_wiggle(seconds=2.0, amplitude=90, speed_s=0.12)
        print('Camera head demo done')
    except Exception as e:
        print('Camera head demo failed:', e)


In [ ]:
import time
try:
    print('Forward / Backward')
    bot.forward(0.6); time.sleep(0.25)
    bot.backward(0.6); time.sleep(0.25)
    bot.stop(); time.sleep(0.1)

    print('Turn left / right')
    bot.turn_left(0.5); time.sleep(0.25)
    bot.turn_right(0.5); time.sleep(0.25)
    bot.stop(); time.sleep(0.1)

    print('Strafe left / right')
    bot.left(0.6); time.sleep(0.25)
    bot.right(0.6); time.sleep(0.25)
    bot.stop(); time.sleep(0.1)

    print('Diagonals')
    bot.diagonal_left(0.6); time.sleep(0.25)
    bot.diagonal_right(0.6); time.sleep(0.25)
    bot.stop(); time.sleep(0.1)

    print('Movement demo done')
except Exception as e:
    bot.stop()
    print('Movement failed:', e)


In [ ]:
import time
try:
    print('Drift left / right')
    bot.drift_left(seconds=0.9, turn_blend=0.55)
    time.sleep(0.2)
    bot.drift_right(seconds=0.9, turn_blend=0.55)
    time.sleep(0.2)
    bot.stop()
    print('Drift demo done')
except Exception as e:
    bot.stop()
    print('Drift failed:', e)


In [ ]:
try:
    print('Horn: meep meep')
    bot.horn(block=True)
    print('Horn done')
except Exception as e:
    print('Horn failed:', e)


In [ ]:
try:
    bot.stop()
except Exception:
    pass
try:
    if eyes:
        eyes.off()
except Exception:
    pass
print('All done')
